In [1]:
import os, csv, json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# CPFA Hypertune Study - Final Analysis

In [2]:
filenames = [f for f in os.listdir('../../data/raw') if f[-4:] == 'json']
filenames

['db_backup3.json', 'db_backup.json', 'db_backup2.json']

###### trials to keep:
cpfa_ht_trial_20191111_081725
cpfa_ht_trial_20191111_080838
cpfa_ht_trial_20191111_080831

In [3]:
results = {}
keeper_trials = {'cpfa_ht_trial_20191111_081725': [], 
                 'cpfa_ht_trial_20191111_080838': [], 
                 'cpfa_ht_trial_20191111_080831': []}

for trial in keeper_trials:
    for i in range(3001):
        keeper_trials[trial].append({})

sample object:
```{"logName":"projects/cpfa-hypertune/logs/master-replica-0","resource":{"type":"ml_job","labels":{"job_id":"cpfa_ht_trial_20191110_115124","project_id":"cpfa-hypertune","task_name":"master-replica-0"}},"jsonPayload":{"message":"food distribution 0\n","levelname":"INFO"},"timestamp":"2019-11-10 22:52:31.313637 UTC","receiveTimestamp":"2019-11-10 22:52:35.761271 UTC","severity":"INFO","insertId":"5u96i6fa2o7si","labels":{"compute_googleapis_com_resource_id":"3176392964891290663","compute_googleapis_com_zone":"us-central1-a","ml_googleapis_com_trial_id":"336","compute_googleapis_com_resource_name":"cmle-training-805699422387418271","ml_googleapis_com_job_id_log_area":"root"}}```

In [4]:
trialcount = 0;
rowsList = []
trials = []
for file in filenames:
    with open(os.path.join('../../data/raw', file)) as json_file:
        for i, line in enumerate(json_file):
            x = json.loads(line)
            job_id = x['resource']['labels']['job_id']
            trialId = int(x['labels']['ml_googleapis_com_trial_id'])
            message = x['jsonPayload']['message']
            if job_id in keeper_trials:
                if 'trialId' not in keeper_trials[job_id][trialId]:
                    keeper_trials[job_id][trialId]['trialId'] = trialId
                if 'Fitness Score' in message:
                    keeper_trials[job_id][trialId]['fitnessScore'] = float(message.split()[3])
                if 'foodDist' not in keeper_trials[job_id][trialId] and 'food' in message:
                    keeper_trials[job_id][trialId]['foodDist'] = float(message.split()[2])
                if 'Fitness' in message:
                    keeper_trials[job_id][trialId]['fitness'] = float(message.split()[3])
                if 'RateOfLayingPheromone' in message:
                    keeper_trials[job_id][trialId]['RateOfLayingPheromone'] = float(message.split()[1])
                if 'ProbabilityOfSwitchingToSearching' in message:
                    keeper_trials[job_id][trialId]['ProbabilityOfSwitchingToSearching'] = float(message.split()[1])
                if 'ProbabilityOfReturningToNest' in message:
                    keeper_trials[job_id][trialId]['ProbabilityOfReturningToNest'] = float(message.split()[1])
                if 'RateOfInformedSearchDecay' in message:
                    keeper_trials[job_id][trialId]['RateOfInformedSearchDecay'] = float(message.split()[1])
                if ('RateOfSiteFidelity' in message):
                    keeper_trials[job_id][trialId]['RateOfSiteFidelity'] = float(message.split()[1])
                if ('RateOfPheromoneDecay' in message):
                    keeper_trials[job_id][trialId]['RateOfPheromoneDecay'] = float(message.split()[1])
                if ('UninformedSearchVariation' in message):
                    keeper_trials[job_id][trialId]['UninformedSearchVariation'] = float(message.split()[1])
            else:
                pass

In [5]:
for trial in keeper_trials:
    df = pd.DataFrame(keeper_trials[trial])
    print(len(df.dropna()))
    df.dropna().sort_values(by=['fitnessScore'], ascending=True).to_csv('{}.csv'.format(trial))

2973
2968
2983


In [7]:
df.dropna().sort_values(by=['fitnessScore'], ascending=False)

,trialId,foodDist,ProbabilityOfReturningToNest,RateOfInformedSearchDecay,RateOfSiteFidelity,RateOfLayingPheromone,fitnessScore,fitness,RateOfPheromoneDecay,UninformedSearchVariation,ProbabilityOfSwitchingToSearching
1753,1753.0,1.0,1.768838e-14,0.146402,0.824976,16.585013,0.762109,0.762109,9.952540,10.936052,0.993863
1744,1744.0,1.0,1.793069e-14,0.112047,0.798555,16.638649,0.759896,0.759896,9.950276,10.925143,0.993136
1355,1355.0,1.0,1.814419e-14,0.105334,0.799547,16.669323,0.759375,0.759375,9.950302,10.916592,0.992329
2097,2097.0,1.0,1.816278e-14,0.126467,0.806915,16.665791,0.758854,0.758854,9.950349,10.916895,0.992657
2549,2549.0,1.0,1.830933e-14,0.119411,0.800288,16.669170,0.758333,0.758333,9.949010,10.920558,0.991907
...,...,...,...,...,...,...,...,...,...,...,...
354,354.0,1.0,8.694984e-01,4.103552,18.359432,1.565932,0.000260,0.000260,9.982856,0.718718,0.979182
213,213.0,1.0,6.190489e-01,5.000000,0.946994,18.056316,0.000260,0.000260,10.000000,11.969257,0.947105
1024,1024.0,1.0,7.276202e-01,5.000000,1.444500,18.254973,0.000000,0.000000,6.188455,0.859613,0.898710
610,610.0,1.0,5.081578e-01,5.000000,12.138746,19.262273,0.000000,0.000000,0.383000,11.926188,0.993237
